# WARNING: this is work in progress, results may change

To do:
- clean up
- test for normality before using posthoc ttest

In [1]:
import scikit_posthocs as sp
import pandas as pd
import pickle

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
with open("../../results/anova_per_patient.pickle", "rb") as handle:
     anova_dict = pickle.load(handle)

<IPython.core.display.Javascript object>

## Select DBS configurations to use

In [3]:
configs = {
    "frequencies": ["OFF", "030", "085", "130"],
    "intensities": ["OFF", "033", "066", "100"],
    "pulse_widths": ["OFF", "040", "090"],
}
dbs_parameter = "frequencies"
selected_configs = configs[dbs_parameter]
print(
    f"Configurations that will be used for ANOVA: {dbs_parameter} ({', '.join(selected_configs)})"
)
print(
    f"We are using the frequencies here, but we can do the same for pulse width or intensity, if necessary."
)

Configurations that will be used for ANOVA: frequencies (OFF, 030, 085, 130)
We are using the frequencies here, but we can do the same for pulse width or intensity, if necessary.


<IPython.core.display.Javascript object>

## Load data (not visible in html)

In [4]:
stride_params = pd.read_csv("../../data/stride_params.csv")
stride_params.drop("time_stamp_s", axis=1, inplace=True)

df = stride_params.set_index("configuration")
df = df.loc[selected_configs]
df = df.reset_index()
config = df["configuration"]
df.drop("configuration", axis=1, inplace=True)
df.insert(1, "configuration", config)

stride_params = df

stride_params[
    (stride_params["patient_id"] == "Pat_03")
    & (stride_params["configuration"] == "040")
]
stride_params.drop("preferred", axis=1, inplace=True)
available_gait_parameters = stride_params.columns[5:]
strides_dict = {
    patient: stride_params[stride_params["patient_id"] == patient]
    for patient in set(stride_params["patient_id"])
}

<IPython.core.display.Javascript object>

In [5]:
# https://pythonrepo.com/repo/maximtrp-scikit-posthocs-python-science-and-data-analysis

gait_test = "free"
gait_parameter = "stride_length_cm"
#gait_parameter = "max_sensor_lift_cm"
anovas = anova_dict[gait_test][gait_parameter]
posthocs = {}
no_difference_feet = []

for patient in anovas.index:
    # posthocs[patient] = pd.DataFrame()
    posthocs[patient] = {}
    df = stride_params[stride_params["patient_id"]==patient]
    for column in anovas.columns:
        foot = column[1]
        #if all(anovas.loc[patient] < 0.05):
        if anovas.at[patient, column] < 0.05:
            params_foot = df[(df['test']==gait_test)]# & (df["foot"] == foot)]
            try:
                posthocs[patient][foot] = sp.posthoc_ttest(params_foot, val_col=gait_parameter, group_col="configuration", p_adjust="holm")              
            except ZeroDivisionError:
                continue
            #posthocs[patient] = sp.posthoc_ttest(params_foot, val_col="stride_length_cm", group_col="configuration", p_adjust="holm")              
        else:
            no_difference_feet.append((patient, column))

<IPython.core.display.Javascript object>

In [6]:
no_off_feet = []
ttest_results = {}

unusable_ttests = pd.DataFrame(columns=[c for c in selected_configs if c!="OFF"])
for patient in posthocs.keys():
    ttest_results[patient] = {}
    for foot in posthocs[patient].keys():
        if not "OFF" in posthocs[patient][foot].index:
            no_off_feet.append((patient, foot))
            continue
        results = posthocs[patient][foot]["OFF"].drop("OFF")
        significant_results = results[results<0.05]
        ttest_results[patient][foot] = significant_results
    if not ttest_results[patient]:
        ttest_results.pop(patient)

<IPython.core.display.Javascript object>

In [7]:
def get_change(data_patient: pd.DataFrame, config:str, test: str, gait_para: str) -> int:
    """Find the change from OFF to config in the given gait parameter.
    
    Parameters
    ----------
    data_patient
        all gait parameters for all strides of a specific patient
    
    config
        something like "085"
        
    test
        one of the used walking tests, like slow, fast, free,...
        
    gait_parameter
        something like `stride_length_cm`
        
    Returns
    -------
    result
        difference (OFF - config)
    """
    
    data_config = data_patient[(data_patient["test"]==test) & (data_patient["configuration"]==config)][gait_para]
    data_off = data_patient[(data_patient["test"]==test) & (data_patient["configuration"]=="OFF")][gait_para]
    return data_config.mean() - data_off.mean()

<IPython.core.display.Javascript object>

In [8]:
changes_overview = pd.DataFrame()
configs = columns=[c for c in selected_configs if c!="OFF"]
for patient, data in ttest_results.items():
    #print(f"\n\n {patient}")
    for foot, foot_data in data.items():   
        #print(f"\n {foot}")
        change = {}
        for config in configs:#foot_data.index:
            if foot_data.get(config, False):
                df = stride_params[(stride_params["patient_id"]==patient) & (stride_params["foot"]==foot)]
                change[config] = get_change(df, config, gait_test, gait_parameter)
            else:
                change[config] = None
            #changes_overview.at[(patient, foot)][config] = result
        idx = pd.MultiIndex.from_tuples([(patient, foot)], names=["patient", "foot"])
        d = pd.DataFrame.from_dict(change, orient='index', columns=idx).T
        changes_overview = changes_overview.append(d)

<IPython.core.display.Javascript object>

# Post hoc

In [9]:
def style_negative(v, props=''):
    return props if v < 0 else None

s2 = changes_overview.style.applymap(style_negative, props='color:red;')\
              .applymap(lambda v: 'opacity: 20%;' if (v < 0) and (v > -0.3) else None)
print("nan = nicht signifikant laut posthoc analyse")
print(f"Werte = Änderung von OFF im Vergleich zur DBS Einstellung bezüglich des Parameters {gait_parameter}")
print(f"--> positiver Wert, z.b. 3.6, bei 085 bedeutet, dass {gait_parameter} bei 085 um 3.6 {gait_parameter.split('_')[-1]} höher war.")
s2

nan = nicht signifikant laut posthoc analyse
Werte = Änderung von OFF im Vergleich zur DBS Einstellung bezüglich des Parameters stride_length_cm
--> positiver Wert, z.b. 3.6, bei 085 bedeutet, dass stride_length_cm bei 085 um 3.6 cm höher war.


<IPython.core.display.Javascript object>

In [10]:
print(gait_parameter)
for config in changes_overview.columns:
    feet_decreased = changes_overview[config][changes_overview[config]<0]
    feet_increased = changes_overview[config][changes_overview[config]>0]
    print(f"Mean deacrease for patients who decreased (N={len(feet_decreased)}) for {config}: {feet_decreased.mean()} ")
    print(f"Mean increase for patients who increased (N={len(feet_increased)}) for {config}: {feet_increased.mean()}")
    print()

stride_length_cm
Mean deacrease for patients who decreased (N=10) for 030: -8.036684253248621 
Mean increase for patients who increased (N=13) for 030: 8.183557246383371

Mean deacrease for patients who decreased (N=7) for 085: -8.125601854587162 
Mean increase for patients who increased (N=18) for 085: 11.997280651427182

Mean deacrease for patients who decreased (N=6) for 130: -7.534440871491715 
Mean increase for patients who increased (N=16) for 130: 9.589410730524271



<IPython.core.display.Javascript object>

In [11]:
print("Mean changes across all feet and patients:\n", changes_overview.mean())

Mean changes across all feet and patients:
 030    1.131278
085    6.362874
130    4.919269
dtype: float64


<IPython.core.display.Javascript object>

In [12]:
test_params = stride_params[stride_params["test"]=="free"]
test_params_mean = test_params.groupby(["patient_id", "configuration"]).mean()
test_params_mean['configuration']=test_params_mean.index.get_level_values(level=1)
test_params_mean.index = test_params_mean.index.droplevel(level=1)

<IPython.core.display.Javascript object>

In [13]:
#sp.posthoc_ttest(test_params, val_col=gait_parameter, group_col='configuration', p_adjust='holm').loc["OFF"]

<IPython.core.display.Javascript object>

In [14]:
#lm = sfa.ols(f"{gait_parameter} ~ C(configuration)", data=test_params).fit()
#anova = sa.stats.anova_lm(lm)
#print(anova)  

<IPython.core.display.Javascript object>